# Progressive Sampling
A common challenge for computing doi values progressively is that the dataset processed so far will eventually become too large, causing the doi computation too take too long between chunks.
The quick solution is to maintain a representative sample of the processed data that is of a constant maximum size, on which the computation is run instead. 
The problem here is that doi values are often unevenly distributed, which means that a random sampling might not catch those "rare" items that the user is actually interested in.

The general idea we follow here is to divide the data into bins of equal density, based on their doi value. 
Then, we progressively maintain a representative set of points for each bin, to be able to capture the distributions of the entire dataset whenever we recompute the doi function or train the classifier.

This task is split into the following subtasks: 
1. compute the doi function over all items across all bins
2. rebin the data based on their doi values
3. update items in bins that have changed "too much"

## 1) Compute the doi function
For simplicity, we will use outlier detection here, which for every item computes, whether or not it is an outlier in the dataset, given an approximate "contamination" of the data.

In [ ]:
import numpy as np
import pandas as pd
import os
from sys import path
cwd = os.getcwd()
path.append(f"{cwd}/..")

from doi_component.outlierness_component import OutliernessComponent

doi_comp = OutliernessComponent()

def doi(X: np.ndarray):
  df = pd.DataFrame(X)
  return doi_comp.compute_doi(df)

## 2) Binning
Given a distribution of floating point values, progressively maintain bins of equal density for them.

In [ ]:
import numpy as np
from random import random
from sklearn.datasets import make_blobs
from sklearn.preprocessing import KBinsDiscretizer
import matplotlib.pyplot as plt

N = 10000
training_size = 100
features = 2
chunk_size = 1000
n_bins = 5

blobs_params = {"n_samples": N, "n_features": features}
hist_params = {"alpha": 0.73, "ec": "#000"}

binning = KBinsDiscretizer(n_bins=n_bins, encode="ordinal", strategy="uniform")
X = make_blobs(centers=6, cluster_std=1, **blobs_params)[0]
y = doi(X)

distributions = [np.random.normal(random(), random() * 0.25, N) for _ in range(N//chunk_size)]
previous_hist = []

for i, distribution in enumerate(distributions):
  y_ = y + distribution

  binning.fit(y_.reshape(-1, 1))
  edges = binning.bin_edges_[0]
  bins = binning.transform(y_.reshape(-1, 1))
  bins = binning.inverse_transform(bins)

  fig, (ax1, ax2) = plt.subplots(2, 1)

  hist = np.histogram(y_, bins=edges)[0]
  if i > 0:
    delta_hist = hist - previous_hist
    ax2.bar(np.arange(0, n_bins), delta_hist, fc="orange", **hist_params)
    ax2.axhline(0, lw=1, c="black")
    ax2.axhline(N * 0.1, lw=1, c="black", dashes=[2])
    ax2.axhline(-N * 0.1, lw=1, c="black", dashes=[2])
    ax2.set_ylim([-N, N])
  previous_hist = hist

  ax1.set_title("chunk")
  ax1.set_ylim([0, N])
  ax1.hist(bins, bins=edges, fc="steelblue", **hist_params)
  ax1.hist(y_, fc="white", **hist_params)

## 3) Sampling
A good candidate seems to be so-called reservoir sampling, which progressively updates a representative sample of the data seen so far, thus at every point maintainnig a sample where each point may have been picked with probability := 1/(number of points processed so far).

See [https://dl.acm.org/doi/pdf/10.1145/3147.3165](https://dl.acm.org/doi/pdf/10.1145/3147.3165)

### 3.1 Naive chunk-based Reservoir sampling
Pseudo code:
```
# we progress over a dataset X of size N and want to maintain a progressive random sample S of 
# size n.
# Every item x_t from the dataset is numbered based on the timepoint t at which it is 
# processed: x_t with t \in [0 ... N]. We go through the data in linear order.
# for x_t in X:
  # if t <= n, then x_t is put into S.
  # otherwise, x_t becomes a candidate that may replace an item in S:
  #    generate a random integer r from the interval [0, t]. If r < n, then replace the item S[r] 
  #    with with x_t
# the probability of an item becoming part of the sample S is n/t and is decreasing with t.
# the probability of being replaced is higher, the longer an item is part of Sy.
```

In [ ]:
from sklearn.utils.random import sample_without_replacement
from random import randint
import numpy as np

reservoir = np.array([])
k = 5

X = np.arange(0, 1000)
chunk_size = 10

for i in range(0, len(X) // chunk_size):
  chunk = X[i * chunk_size:(i+1) * chunk_size]

  if i == 0:
    sample = sample_without_replacement(n_population=chunk_size, n_samples=k, random_state=i)
    reservoir = chunk[sample]
  else:
    candidates = np.array([randint(0, i*chunk_size) for _ in range(chunk_size)])
    picks = candidates[candidates < k]
    reservoir[picks] = chunk[picks]

reservoir

### 3.2 Optimized chunk-based Reservoir Sampling

Quote from the reservoir sampling paper:
> The random variable S(n, t) is defined to be the number of records in the file that are skipped over before the next record is chosen for the reservoir, where n is the size of the sample and where t is the number of records processed so far. For notational simplicity, we shall often write 9 for S(n, t),in which case the parameters n and t will be implicit.

In [ ]:
from sklearn.utils.random import sample_without_replacement
from random import randint
import numpy as np

reservoir = np.array([])
k = 5

X = np.arange(0, 1000)
chunk_size = 10

skip = k

for i in range(0, len(X) // chunk_size):
  chunk = X[i * chunk_size:(i+1) * chunk_size]

  if i == 0:
    sample = sample_without_replacement(n_population=chunk_size, n_samples=k, random_state=i)
    reservoir = chunk[sample]
    skip = randint(k, (i+1)*chunk_size)
  elif skip < i*chunk_size:
    candidate = chunk[skip % chunk_size]
    reservoir[randint(0, k-1)] = candidate
    skip = randint(k, i*chunk_size*2)

reservoir

## 4) Progressive Bin-based Sampling with Balanced Classes
Putting things together.

In [ ]:
def plot_bar():
  # plot the results
  fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(7, 7), gridspec_kw={"hspace": 1.25})
  ax1.set_title("ground truth y values")
  ax1.hist(y, **hist_params)

  ax3.set_title("doi computed from sampled X")
  reservoir = np.concatenate(tuple([bin_sampler.X_reservoir[x] for x in bin_sampler.X_reservoir]))
  sampled_X_y = doi(reservoir)
  ax3.hist(sampled_X_y, **hist_params)

def plot_scatter():
  reservoir_X = np.concatenate(tuple([bin_sampler.X_reservoir[x] for x in bin_sampler.X_reservoir]))
  reservoir_y = doi(reservoir_X)
  chunk_y = doi(chunk)
  fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 10))
  ax1.set_title("reservoir")
  ax1.scatter(x=reservoir_X[:,0], y=reservoir_X[:,1], c=reservoir_y, alpha=0.1)
  ax2.set_title("chunk")
  ax2.scatter(x=chunk[:,0], y=chunk[:,1], c=chunk_y, alpha=0.1)

In [ ]:
def rebin_reservoirs(processed):
  '''
  Recompute the doi function over all items in the reservoir, then redistribute the data into bins 
  again.
  '''
  X_ = bin_sampler.get_current_sample()
  y_ = doi(X_)

  bin_sampler.reset_reservoirs()

  return bin_sampler.add_chunk(X_, y_, processed)

In [ ]:
from random import randint
import numpy as np
from random import random
from sklearn.datasets import make_blobs
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.utils.random import sample_without_replacement
import matplotlib.pyplot as plt
from storage_strategy.progressive_bin_sampler import ProgressiveBinSampler

N = 10000
features = 2
chunk_size = 1000

chunks = N // chunk_size

blobs_params = {"n_samples": N, "n_features": features}
hist_params = {"alpha": 0.73, "ec": "#000"}
binning = KBinsDiscretizer(n_bins=n_bins, encode="ordinal", strategy="kmeans")

X = make_blobs(centers=6, cluster_std=1, **blobs_params)[0]
y = doi(X)

# last iteration's per-bin mean values
previous_bin_means = np.array([])

# sum of squared differences betwen partial doi and full doi computation
mse = []

bin_sampler = ProgressiveBinSampler()

for i in range(chunks):
  print("\n#chunk:", i)

  chunk = X[i*chunk_size:(i+1)*chunk_size]

  # compute the doi function over known items and the items in the new chunk
  progressive_sample = bin_sampler.get_current_sample()
  X_ = np.append(progressive_sample, chunk, axis=0)
  y_ = doi(X_)

  bin_sampler.add_chunk(X_[-chunk_size:], y_[-chunk_size:], i*chunk_size)


  # TODO: check if rebinning is necessary
  # compute the prediction error for each chunk
  error = np.mean((y_[-chunk_size:]-y[i*chunk_size:(i+1)*chunk_size])**2)
  mse += [error]

  # rebin every 4 iterations
  if i > 0 and i % 4 == 0:
    rebin_reservoirs(i*chunk_size)
  
  plot_scatter()

print("error progression:", mse)
plot_bar()